In [20]:
import numpy as np
import math 
import random

from google.colab import files
files.upload()

def generate_quadgram_mapping():
  frequency = {}
  f = open("english_quadgrams.txt", "r")
  lines = f.readlines()
  for line in lines:
    data = line.split(" ")
    frequency[data[0]] = data[1]
  return frequency

def calculate_quadgram_count(ciphertext):
  count = 0
  for i in range(len(ciphertext) - 4):
    count += 1
  return count 

def tetra_graph_score(ciphertext, key, frequency, N):
  score = 0 
  plaintext = decode_playfair(ciphertext, key)
  for i in range(len(plaintext) - 4):
    quadgram = plaintext[i:i+4]
    propability = 0.00001
    if(quadgram in frequency):
      propability = float(frequency[quadgram].strip())
    score = score + math.log(propability / N, 10) 
  return score 

def decode_playfair(text, key):
  plaintext = []

  for i in range(len(text) - 1):
    if i%2 != 0: continue

    first_character_coordinates = np.where(key == text[i])
    first_character_row_index = first_character_coordinates[0][0]
    first_character_col_index = first_character_coordinates[1][0]

    second_character_coordinates = np.where(key == text[i+1])
    second_character_row_index = second_character_coordinates[0][0]
    second_character_col_index = second_character_coordinates[1][0]

    plain_text_first_char = ""
    plain_text_second_char = ""
    if first_character_row_index == second_character_row_index:
      # Same row
      key_row = key[first_character_row_index, :]
      plain_text_first_char = key_row[first_character_col_index - 1]
      plain_text_second_char = key_row[second_character_col_index - 1]
    
    elif first_character_col_index == second_character_col_index:
      # Same col
      key_col = key[:, first_character_col_index]
      plain_text_first_char = key_col[first_character_row_index - 1]
      plain_text_second_char = key_col[second_character_row_index - 1]
    else:
      # rectange, swap opposites
      plain_text_first_char = key[first_character_row_index, second_character_col_index]
      plain_text_second_char = key[second_character_row_index, first_character_col_index]
    
    plaintext.append(plain_text_first_char + plain_text_second_char)
  return "".join(plaintext)

def randomly_generate_key():
  alphabet = "ABCDEFGHIKLMNOPQRSTUVWXYZ" # leaving out J
  random_key_str = ''.join(random.sample(alphabet,len(alphabet)))
  random_key_list = [
      [*random_key_str[0:5]],
      [*random_key_str[5:10]],
      [*random_key_str[10:15]],
      [*random_key_str[15:20]],
      [*random_key_str[20:25]],
  ]
  random_key = np.array(random_key_list)
  return random_key

def randomly_mutate_key(key):

  new_key = np.copy(key)

  if random.randrange(1, 20) < 2:
    # swap rows
    row1 = random.randrange(5)
    row2 = random.randrange(5)
    temp = np.copy(new_key[row1, :])
    new_key[row1, :] = np.copy(new_key[row2, :])
    new_key[row2, :] = temp
    # return new_key

  # Experimentally found that random col swapping didnt help
  if random.randrange(1, 20) < 2:
    # swap cols
    col1 = random.randrange(5)
    col2 = random.randrange(5)
    temp = np.copy(new_key[:, col1])
    new_key[:, col1] = np.copy(new_key[:, col2])
    new_key[:, col2] = temp

  num_swaps = random.randrange(0, 4)
  for _ in range(num_swaps):
    row1 = random.randrange(5)
    row2 = random.randrange(5)
    col1 = random.randrange(5)
    col2 = random.randrange(5)

    temp = new_key[row1][col1]
    new_key[row1][col1] = new_key[row2][col2]
    new_key[row2][col2] = temp

  return new_key


def hillclimbing(ciphertext, test_key, frequency, N, num_tries):
  current_score = tetra_graph_score(ciphertext, test_key, frequency, N)
  current_key = test_key
  for i in range(num_tries):
    new_key = randomly_mutate_key(current_key)
    new_score = tetra_graph_score(cipher_text, new_key, frequency, N)
    incremental_cost = new_score - current_score
    if incremental_cost > 0:
      current_score = new_score
      current_key = new_key
    # else:
    #   mutate_probability = math.exp(incremental_cost/(num_tries - i))
    #   if(random.random() < mutate_probability):
    #     current_score = new_score
    #     current_key = new_key
    if i % 1000 == 0:
      print(i)
  return current_key, current_score

initial_partial_key = [
    ["Z", "M", "E", "H", "B"],
    ["F", "K", "I", "O", "N"],
    ["A", "D", "P", "Q", "R"],
    ["S", "C", "U", "L", "X"],
    ["T", "W", "V", "G", "Y"],
]

random.seed(42)
partial_key = np.array(initial_partial_key)
#main
cipher_text = 'HSYOGZHKGVGOBQCZNTKEMPZQGOVBCWMKVZUMKQZANTWYKQAXACAKGZBWDTPDCZZDPYGZBMDYZNVGATLKBCZDTXSRQKEKSVCVPALKBCZDHSYOGZKQMHSAYZBRCZMHYDZNRKACVGZUEKTAVOGCKHURWTGZTPIUAZTXSDZLSAMRMPLKBCZDNTBYSTLKBCZDQKFCVMAIBMCGYOGOZKCUNXZDPOCZZDPYGZUORSHLLKBCZDNTAIZUVGAYBMARHMGZTPIUAZRBBRBMLKTZTPRHPDCZZDNCTPVGFTZAGZMKKAQZAKNGBMZLSAMRAZGZKQZRQKBMAVNXQHZVLKBCZDAYGCXUVGLCTPTLAGQUCZAFRMDRHXKLCKWNKOCAACIAKAAZBGXKKCMRLKBCZDNTUIRTZRCGYOGOMKSVLNQUGYZGACYOGZKQMKCZZDTPRMMRLKBCZDGDAYAFRMAVBMLKAQKQLNGZHVQKNZZMDZGZMBLARBQKAYMRATLKBCZDYNLNBKZVQBQKAYMDCVFTLAIOCZZDUBCATPDYKGKQUBKZWYGVMFCTTZMRZABRACZAMLCZZDRMRDLZLKBCZDRMRDKDVMSCACAKGZIXCZZDLXRAIMFTBMUENTUAZAVHGZKQTPPMDBSTACYOGZMBLARBHDAYZUEKNUXOKHTAFZKQACYNZUVZEKDAZUXOXFMRAZBMQKYDTAGZTPYVXZMRATMKCZZDARMPGZHKGVGOBQCZNTODCHXGACYZAGFBXNRAUBZRRXMRACAKGZISAZUBBQRAGZBQMDRKSPGVLMLNGZBKZRAYLKBCZDRAYDDTPDCZZDNTWYKQMRGZHKGVGOBQCZNTACYABMZUUBHKURXOOGBMHDPOELTPDYWZATURVHNLDENTBSVBRDDXCZZDZDCVFTGZMHUXRTHQKABMAQGYZGNTSCURISAZTANYBMTSQKUOXUZVQUBKMKCZZDNTURQKVUNZHQFUZATXFGKAGZHQRTKAACYNZLRDZVBMHDPOELTPDYQPDGQKSCACAKGZISAZGZBVRDZVBMQMCNKHFBKXGIMVTZMGUXNTACNGBMZRAYFBKQZVBMPYLCMRPYIOEUFUMRACZNQKTXSRHDPOELTPRARKSCACAKGZISAZTALKBCZDHSVGDZGZMVUQDVZQPDCZZDUOGZLAHLGCGIMVTZBMOUPALNGZZQKAGZACYNBMTZBMBWSNGZBQGVLMLNGZBKZRAYNSAUZRAIBMHDPOELTPWSRBTZRAPYNGHMWSRBFTKQZAHMLKBCZDNTPMODHMNZATKMACVGDZGZBQGVLMLNGZZQKAGZACZAMLCZZDVUDOTCSAKDVMSCACAKGZISAZRDBVLARDZRXFGVLMVBDXCZZDTPRMMRLKBCZDNTWYKQMRGZHKGVGOBQCZNTACYZAGTDZDTLCAPYIOMLCZZDNTAIBMQMCNKHXKEKHMACVGDZTAZNDWXNKQGFOUAYBMPYLCMRPYIOBFRAFBKAFUMRBMQKLKBCZDKAYDZLFUUMRTBRZVGZMQZKFCHQQKMHBWHKPUAZHSVGGZLAHLZNXOXFGVLMBRQBQKPYLCMRLKBCZDARMPGZHVVDXLCZZDFBXNRAZUUGQUGYZGZQMRNZNUDMFTRDZRXFGVLMACZAMLCZZDTALKBCZDGZNZYDUTQKNZCMAYROMRAZGZKQBQKQZKFOGWLASQGVLMBRTPUOOGBMOUPALNAOLKBCZDNTAINXLNGZMHLKBCZDGZXABQZLCGEKTPGZZQKAGZNTAIBMAXAGMLCZZDRDHVKOCAPYIOBMAUAZVCTANYMGDYAYDSMKCZZDNLDVDZATZKNGBMBCBIRBUGLKBCZDNTPDNTTAYZMHYDZNRKPAAZGZXABQZLSPHKVGTZBMZRAYNTAIBMQBAYQBQKLKBCZDTPRMMRLKBCZDPYLCMRPYIOBMWXAGTANYZUVTLAZUEKCVXOKHACQZLNGZMHLKBCZDGZKQMVWXAGATBQMDRKSPGVLMTPUOOGBMOUPALNAOGZZKQZFBKXGIBRTPGZBQBRIAAZHSIOCZZDARMPACAKGZIXCZZDUNVZDRZCHMTZAYZLFHZGLKBCZDRSTZMHZUAYKXPCZVZUYOGZBQGVLMBRQBQKUOCUZRSIZRDANTSIZRDAUTCAAZFBCTARAFVZCMQPDGQKAIBMCGYOGOZKCUNXZDPAAZUBMVDBTSAFYNZLCGEKDYQONTARDXCZZDFBKQMVODHMQUCZACYOGZBQGVLMLNGZZQKAGZRBMHMRZAGZTPLQKANZMCMKCZZDPYGZBMUEZRSMXNMRNTUMKQBVBRDFATGZHVAKAFSCACYOGZZQUBHKKOKCMRDFNTUADTPDCZZDMPATAZOQXNFBKQKRNTARDYDXCZZDNLDVZUEKQZEKUEZRDUGZNZLKAFVAIBZQVGATMRZMNLQKACYZAGVZXOCHMKCZZDYDGVDOGIGCTMMRLKAFVATXMLCZZDVBDXCZZDQKFCVMSCACAKGZISAZGZZQGZNZDOLKAFYANLDVIAAZTPGZMKGIGCTMMRLKAFVAVMDYZMCGKEZVBMQKRDZQXNGVLMBRKHNGLKBCZDTASQTMRDUALKBCZDTADAKAKMQKWSRBDXCZZDTADEUTMDTDWAACYZAGLKBCZDTXSABKMKCZZDGZHKNTACNLEBRAQZEKSEBKZQDMIGCGEMPDCZZDNLSQZRQKLDZAWLWLQUCZUOOGBMQKAYLNGZBQKACU'
frequency = generate_quadgram_mapping()
# key = randomly_generate_key()
count = calculate_quadgram_count(cipher_text)
best_key, best_score = hillclimbing(cipher_text, partial_key, frequency, count, 30000)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000


In [22]:
print(best_key)

[['Z' 'E' 'M' 'H' 'B']
 ['O' 'K' 'C' 'L' 'X']
 ['A' 'R' 'D' 'U' 'S']
 ['N' 'Q' 'P' 'F' 'W']
 ['T' 'V' 'I' 'G' 'Y']]


In [23]:
print(decode_playfair(cipher_text, best_key))

BUTXTHELITTLEWOMANEVIDENTLYEXPECTEDHERTOANSWERSODOROTHYSAIDCOMMAWITHHESITATIONCOMXMAYOUAREVERYKINDCOMXMABUTXTHEREMUSTBESOMEMISTAKEDOTIHAVENOTKILLEDANYTHINGDOTYOURHOUSEDIDCOMXMAANYWAYCOMXMAREPLIEDTHELITXTLEOLDWOMANCOMMAWITHALAUGHCOMXMAANDTHATISTHESAMETHINGDOTSEESHECONTINUEDCOMMAPOINTINGTOTHECORNEROFTHEHOUSEDOTTHEREAREHERTWOFEETCOMXMASTILLSTICKINGOUTFROMUNDERABLOCKOFWOXODDOTDOROTHYLOOKEDCOMXMAANDGAVEALITXTLECRYOFFRIGHTDOTXTHERECOMMAINDEEDCOMXMAIUSTUNDERTHECORNEROFTHEGREATBEAMTHEHOUSERESTEDONCOMXMATWOFEXETWERESTICKINGOUTCOMMASHODINSILVERSHOESWITHPOINTEDTOESDOTOHCOMMADEAROHCOMXMADEARCRIEDXDOROTHYCOMMACLASPINGHERHANDSTOGETHERINDISMAYDOTXTHEHOUSEMUSTHAVEFALXLENONHERDOTWHATEVERSHALXLWEDOTHEREISNOTHINGTOBEDONECOMMASAIDTHELITTLEWOMANCALMLYDOTBUTWHOWASSHEASKEDDOROTHYDOTSHEWASTHEWICKEDWITCHOFTHEXEASTCOMXMAASISAIDCOMMAANSWEREDTHELITTLEWOMANDOTSHEHASHELDALXLTHEMUNCHKINSINBONDAGEFORMANYXYEARSCOMMAMAKINGTHEMSLAVEFORHERNIGHTANDXDAYDOTNOWTHEYAREALLSETFREXECOMMAANDAREGRATEFULTOYOUFORTHEFAVORDOTWHOARETHE